In [1]:
import os
import sys
import random
import time
from random import seed, randint
import argparse
import platform
from datetime import datetime
import imp
import numpy as np
import fileinput
from itertools import product
import pandas as pd
from scipy.interpolate import griddata
from scipy.interpolate import interp2d
import seaborn as sns
from os import listdir

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import griddata
import matplotlib as mpl
sys.path.insert(0,'..')
# from notebookFunctions import *
# from .. import notebookFunctions

%matplotlib inline
plt.rcParams['figure.figsize'] = (10,6.180)    #golden ratio
# %matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '/Users/weilu/openmmawsem/')

from openmmawsem import *

In [3]:
class ForceReporter(object):
    def __init__(self, file, reportInterval):
        self._out = open(file, 'w')
        self._reportInterval = reportInterval

    def __del__(self):
        self._out.close()

    def describeNextReport(self, simulation):
        steps = self._reportInterval - simulation.currentStep%self._reportInterval
        return (steps, False, False, True, False)

    def report(self, simulation, state):
        forces = state.getForces().value_in_unit(kilocalories/mole/nanometer)
        self._out.write(str(state.getTime())+"\n")
        #self._out.write(str(dir(state)))
        for f in forces:
            self._out.write('%g %g %g\n' % (f[0], f[1], f[2]))

In [4]:
# build OpenMMAWSEM simulation object
# check each energy term individually
pdb_id = '2xov'
chain_id = 'A'
# prepare pdb
# res_names = prepare_pdb("%s.pdb" % pdb_id, [chain_id])

def openawsemsystem(pdb='2xov.pdb',chain='A',term=-1):
    resnames=prepare_pdb(pdb,chain)
    output_pdb_filename = pdb.split('.')[0] + '-openmmawsem.pdb'
    oa = OpenMMAWSEMSystem(output_pdb_filename,resnames, xml_filename="../awsem.xml")

    # setup Hamiltonian
    if term==1:
        print('apply_con_term')
        term_name='Con'
        apply_con_term(oa)
    elif term==2:
        print('apply_chain_term')
        term_name='Chain'
        apply_chain_term(oa)
    elif term==3:
        print('apply_chi_term')
        term_name='Chi'
        apply_chi_term(oa)
    elif term==4:
        print('apply_excl_term')
        term_name='Excluded'
        apply_excl_term(oa)
    elif term==5:
        print('apply_rama_term')
        term_name='Rama'
        apply_rama_term(oa)
    elif term==6:
        print('apply_direct_term')
        term_name='Direct'
        apply_direct_term(oa)
    elif term==7:
        print('apply_mediated_term')
        term_name='Mediated'
        apply_mediated_term(oa)
    elif term==8:
        print('apply_contact_term')
        term_name='Contact'
        apply_contact_term(oa)
    elif term==9:
        print('apply_beta_term')
        term_name='Beta'
        apply_beta_term(oa)
    elif term==10:
        print('apply_pap_term')
        term_name='P_AP'
        apply_pap_term(oa)
    elif term==11:
        print('apply_dsb_term')
        term_name='DSB'
        apply_dsb_term(oa)
    elif term==12:
        print('apply_helix_term')
        term_name='HELIX'
        apply_helix_term(oa)
    elif term==13:
        print('apply_associative_memory_term')
        term_name='Frag_Mem'
        apply_associative_memory_term(oa)
    return term_name,oa
import time

OA_energy={}

for i in range(3,4):
    if i==12:
        continue
        #pass
    term_name,oa=openawsemsystem(term=i)
    # setup simulation parameters
    reporter_frequency = 1000
    num_steps = 1000
    step_size = 2e-10*femtoseconds
    temperature = 100*kelvin
#     platform = Platform.getPlatformByName('Reference') # OpenCL, CUDA, CPU, or Reference

    # start simulation
    #integrator = VerletIntegrator(step_size)
    integrator = LangevinIntegrator(temperature, 1/picosecond, step_size)
    #simulation = Simulation(oa.pdb.topology, oa.system, integrator, platform)
    simulation = Simulation(oa.pdb.topology, oa.system, integrator)
    simulation.context.setPositions(oa.pdb.positions)
    #simulation.context.setVelocitiesToTemperature(temperature, 1)
    #simulation.minimizeEnergy(tolerance=Quantity(value=.0001, unit=kilojoule/mole))
    simulation.reporters.append(PDBReporter('output.pdb', reporter_frequency))
    #simulation.reporters.append(ForceReporter('forces.txt', 100))
    simulation.reporters.append(StateDataReporter(stdout, reporter_frequency, step=True,
            potentialEnergy=True, temperature=True))
    tt=time.time()
    simulation.step(1)
    state = simulation.context.getState(getEnergy=True)
#     print((time.time()-tt)*1000,'ms')
#     print(term_name, state.getPotentialEnergy())
    print(term_name, state.getPotentialEnergy().value_in_unit(kilocalories/mole), "kcal/mol")
    OA_energy[term_name]=state.getPotentialEnergy()._value
    #print (i,time.time()-tt)

apply_chi_term
Chi 22.4970494811895 kcal/mol


In [5]:
pd.read_csv("energy.dat").head()

,Steps,Chain,Chi,Rama,Water,Helix,AMH-Go,Frag_Mem,Membrane,VTotal,Ebond,Epair,Rg
0,0,104.187332,22.560808,-684.542272,0,0,-579.196818,0,-33.468043,-1170.458994,204.796601,241.126954,12.613729
1,4000,119.441742,31.616431,-767.291508,0,0,-499.919497,0,-35.712385,-1151.865217,199.662062,12.762483,10.051979
2,8000,106.335053,35.444899,-767.858180,0,0,-497.425184,0,-37.370172,-1160.873583,193.744118,12.744725,8.887938
3,12000,129.448049,30.476499,-752.245557,0,0,-506.456595,0,-37.365454,-1136.143059,196.853812,13.510498,11.008106
4,16000,132.939655,37.302584,-732.136528,0,0,-498.904816,0,-37.092766,-1097.891871,204.076576,20.527288,9.731892


In [6]:
data = pd.read_table("2xov-openmmawsem.pdb", sep="\s+", header=None, names=["_","i","type","res","chain","res_id", "x","y","z","_","_","_"])

In [7]:
data = data.dropna().reset_index()
data["res_id"] = data["res_id"].apply(lambda x: int(x))

In [8]:
chosen = data.query("res != 'IGL' and res_id != 1 and res_id != 181")

In [9]:
ca_all = chosen.query("type == 'CA'")[["x","y","z"]]

In [10]:
ca_all.shape

(159, 3)

In [11]:
def compute_chi(data):
    ca_all = data.query("type == 'CA'")[["x","y","z"]].values
    cb_all = data.query("type == 'CB'")[["x","y","z"]].values
    c_all = data.query("type == 'C'")[["x","y","z"]].values
    n_all = data.query("type == 'N'")[["x","y","z"]].values
    energy = 0 
    for i in range(len(n_all)):
        ca = ca_all[i]
        cb = cb_all[i]
        c = c_all[i]
        n = n_all[i]
        chi0 = -0.71
        k_chi = 60*4.184
        r_ca_cb = cb-ca
        r_c_ca = ca-c
        r_ca_n = n-ca
        norm_r_ca_cb = np.sum(r_ca_cb**2)**0.5
        norm_r_c_ca = np.sum(r_c_ca**2)**0.5
        norm_r_ca_n = np.sum(r_ca_n**2)**0.5
        a = np.cross(-r_c_ca,r_ca_n)/norm_r_c_ca/norm_r_ca_n
        chi = np.dot(a,r_ca_cb)/norm_r_ca_cb
        dchi = chi - chi0
        energy += k_chi*dchi*dchi
    return energy

In [12]:
compute_chi(chosen)/4.184

22.490468654328367

In [13]:
ca = data.query("type == 'CA'")[["x","y","z"]]
cb = data.query("type == 'CB'")[["x","y","z"]]
c = data.query("type == 'C'")[["x","y","z"]]
n = data.query("type == 'N'")[["x","y","z"]]
ca = ca.values[1]
cb = cb.values[1]
c = c.values[1]
n = n.values[0]

In [14]:
chi0 = -0.71
k_chi = 60*4.184
r_ca_cb = cb-ca
r_c_ca = ca-c
r_ca_n = n-ca
norm_r_ca_cb = np.sum(r_ca_cb**2)**0.5
norm_r_c_ca = np.sum(r_c_ca**2)**0.5
norm_r_ca_n = np.sum(r_ca_n**2)**0.5
a = np.cross(r_c_ca,r_ca_n)/norm_r_c_ca/norm_r_ca_n
chi = np.dot(a,r_ca_cb)/norm_r_ca_cb
dchi = chi - chi0

In [15]:
chi0 = -0.71
k_chi = 60*4.184
r_ca_cb = ca-cb
r_c_ca = c-ca
r_ca_n = ca-n
norm_r_ca_cb = np.sum(r_ca_cb**2)**0.5
norm_r_c_ca = np.sum(r_c_ca**2)**0.5
norm_r_ca_n = np.sum(r_ca_n**2)**0.5
a = np.cross(r_c_ca,r_ca_n)/norm_r_c_ca/norm_r_ca_n
chi = np.dot(a,r_ca_cb)/norm_r_ca_cb
dchi = chi - chi0

In [16]:
chi

-0.746560419549821

In [17]:
def compute_chi(ca,cb,c,n):
    chi0 = -0.71
    k_chi = 60*4.184
    r_ca_cb = cb-ca
    r_c_ca = ca-c
    r_ca_n = n-ca
    norm_r_ca_cb = np.sum(r_ca_cb**2)**0.5
    norm_r_c_ca = np.sum(r_c_ca**2)**0.5
    norm_r_ca_n = np.sum(r_ca_n**2)**0.5
    a = np.cross(-r_c_ca,r_ca_n)/norm_r_c_ca/norm_r_ca_n
    chi = np.dot(a,r_ca_cb)/norm_r_ca_cb
    return k_chi*dchi*dchi

In [18]:
compute_chi(ca,cb,c,n)

0.3355562002634993